# Set up

In [1]:
# Importing Necessary Packages
import numpy as np
import pandas as pd
import sqlite3
from faker import Faker
import random
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

supply_chain_database_path = os.getenv("supply_chain_database_path")

In [3]:
conn = sqlite3.connect(supply_chain_database_path)

df = pd.read_sql_query("SELECT * FROM raw_order_data", conn)

In [4]:
# Set the seed for reproducibility
SEED = 42
fake = Faker()
fake.seed_instance(SEED) 
random.seed(42)

# 1. Data Overview
understanding our data, checking for missing values, understanding the data distribution and handle missing data or outliers

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

## Numerical Data

- there are 3 missing customer zipcodes
- product description is empty
- order zipcodes are mostly empty as well
- Product Card Id is the same as Order Item Cardprod Id
- Order Customer Id is the same as Customer Id
- Category Id is the same as Product Category Id

In [6]:
pd.set_option('display.max_columns', None)  # Display all columns

# numerical_variable_summary = supply_chain_df.describe()
numerical_df = df.select_dtypes(include=['int64', 'float64'])
numerical_variable_summary = numerical_df.describe()

print(numerical_variable_summary)

       Days for shipping (real)  Days for shipment (scheduled)  \
count             180519.000000                  180519.000000   
mean                   3.497654                       2.931847   
std                    1.623722                       1.374449   
min                    0.000000                       0.000000   
25%                    2.000000                       2.000000   
50%                    3.000000                       4.000000   
75%                    5.000000                       4.000000   
max                    6.000000                       4.000000   

       Benefit per order  Sales per customer  Late_delivery_risk  \
count      180519.000000       180519.000000       180519.000000   
mean           21.974989          183.107609            0.548291   
std           104.433526          120.043670            0.497664   
min         -4274.979980            7.490000            0.000000   
25%             7.000000          104.379997            0.000000 

In [7]:
# Total number of rows in the DataFrame
total_rows = len(df)

# Columns with less counts than the total number of rows
columns_with_missing_values = numerical_df.columns[numerical_df.isnull().sum() > 0]

print("Columns with missing values:")
print(columns_with_missing_values)

Columns with missing values:
Index(['Customer Zipcode', 'Order Zipcode'], dtype='object')


In [8]:
# Check if Product Card Id is equal to Order Item Cardprod Id
product_card_id_match = (df['Product Card Id'] == df['Order Item Cardprod Id']).all()

# Check if Order Customer Id is equal to Customer Id
order_customer_id_match = (df['Order Customer Id'] == df['Customer Id']).all()

# Check if Product Category Id is equal to Category Id
product_category_id_match = (df['Product Category Id'] == df['Category Id']).all()

# Check if Benefit per order is equal to Order Profit Per Order
order_profit_match = (df['Benefit per order'] == df['Order Profit Per Order']).all()

# Check if Product Price is equal to Order Item Product Price
product_price_match = (df['Order Item Product Price'] == df['Product Price']).all()

# Print the results
print(f"Product Card Id matches Order Item Cardprod Id: {product_card_id_match}")
print(f"Order Customer Id matches Customer Id: {order_customer_id_match}")
print(f"Product Category Id matches Category Id: {product_category_id_match}")  
print(f"Benefit per order matches Order Profit Per Order: {order_profit_match}")
print(f"Product Price matches Order Item Product Price: {product_price_match}")

Product Card Id matches Order Item Cardprod Id: True
Order Customer Id matches Customer Id: True
Product Category Id matches Category Id: True
Benefit per order matches Order Profit Per Order: True
Product Price matches Order Item Product Price: True


## Categorical Data
- there are 8 missing customer last names
- customer email and customer password are masked as "XXXXXXXXX"

In [9]:
pd.set_option('display.max_columns', None)  # Display all columns

categorical_df = df.select_dtypes(include=['object'])
categorical_variable_summary = categorical_df.describe()
print(categorical_variable_summary)

          Type Delivery Status Category Name Customer City Customer Country  \
count   180519          180519        180519        180519           180519   
unique       4               4            50           563                2   
top      DEBIT   Late delivery        Cleats        Caguas          EE. UU.   
freq     69295           98977         24551         66770           111146   

       Customer Email Customer Fname Customer Lname Customer Password  \
count          180519         180519         180511            180519   
unique              1            782           1109                 1   
top         XXXXXXXXX           Mary          Smith         XXXXXXXXX   
freq           180519          65150          64104            180519   

       Customer Segment Customer State          Customer Street  \
count            180519         180519                   180519   
unique                3             46                     7458   
top            Consumer             P

In [10]:
# Total number of rows in the DataFrame
total_rows = len(df)

# Columns with less counts than the total number of rows
columns_with_missing_values = categorical_df.columns[categorical_df.isnull().sum() > 0]

print("Columns with missing values:")
print(columns_with_missing_values)

Columns with missing values:
Index(['Customer Lname', 'Product Description'], dtype='object')


In [11]:
# Find columns with only one unique value
columns_with_one_unique_value = categorical_df.columns[categorical_df.nunique() == 1]

print("Columns with only one unique value:")
for column in columns_with_one_unique_value:
    print(f"{column}: {categorical_df[column].unique()[0]}")

Columns with only one unique value:
Customer Email: XXXXXXXXX
Customer Password: XXXXXXXXX


# 2. Data Cleaning

dropping redundant columns

In [12]:
columns_to_drop = ['Customer Zipcode', 'Product Description', 'Order Zipcode', 'Customer Email', 'Customer Password', 'Order Item Cardprod Id', 'Order Customer Id', 'Category Id', 'Benefit per order', 'Order Item Product Price', 'Product Image']

# Drop columns
df.drop(columns=columns_to_drop, inplace=True)

Renaming columns and country names

In [13]:
# Renaming columns appropriately
cleaned_df = df.rename(columns={
    'Type': 'Transaction Type',
    'Days for shipping (real)': 'Days for shipment (real)',
    'Category Name': 'Product Category',
    'Department Name': 'Warehouse Name', 
    'Department Id' : 'Warehouse Id',     
    'Latitude' : 'Warehouse Latitude',
    'Longitude' : 'Warehouse Longitude',
    'order date (DateOrders)': 'Order Date',
    'Order Profit Per Order': 'Order Profit',
    'Product Card Id': 'Product Id',
    'shipping date (DateOrders)': 'Shipping Date',
    'Order Item Quantity': 'Total Quantity Purchased'
})

In [14]:
# Editing the 'Order Country' names
cleaned_df['Product Name'] = cleaned_df['Product Name'].str.strip()
country_mapping = {
    'Japón': 'Japan',
    'Corea del Sur': 'South Korea',
    'Singapur': 'Singapore',
    'Turquía': 'Turkey',
    'Estados Unidos': 'United States',
    'República Democrática del Congo': 'Democratic Republic of the Congo',
    'Senegal': 'Senegal',
    'Marruecos': 'Morocco',
    'Alemania': 'Germany',
    'Francia': 'France',
    'Países Bajos': 'Netherlands',
    'Reino Unido': 'United Kingdom',
    'Guatemala': 'Guatemala',
    'El Salvador': 'El Salvador',
    'Panamá': 'Panama',
    'República Dominicana': 'Dominican Republic',
    'Venezuela': 'Venezuela',
    'Colombia': 'Colombia',
    'Honduras': 'Honduras',
    'Brasil': 'Brazil',
    'México': 'Mexico',
    'Uruguay': 'Uruguay',
    'Argentina': 'Argentina',
    'Cuba': 'Cuba',
    'Perú': 'Peru',
    'Nicaragua': 'Nicaragua',
    'Ecuador': 'Ecuador',
    'Angola': 'Angola',
    'Sudán': 'Sudan',
    'Somalia': 'Somalia',
    'Costa de Marfil': 'Ivory Coast',
    'Egipto': 'Egypt',
    'España': 'Spain',
    'Suecia': 'Sweden',
    'Austria': 'Austria',
    'Canada': 'Canada',
    'Madagascar': 'Madagascar',
    'Argelia': 'Algeria',
    'Liberia': 'Liberia',
    'Zambia': 'Zambia',
    'Níger': 'Niger',
    'SudAfrica': 'South Africa',
    'Mozambique': 'Mozambique',
    'Tanzania': 'Tanzania',
    'Ruanda': 'Rwanda',
    'Israel': 'Israel',
    'Nueva Zelanda': 'New Zealand',
    'Bangladés': 'Bangladesh',
    'Tailandia': 'Thailand',
    'Irak': 'Iraq',
    'Arabia Saudí': 'Saudi Arabia',
    'Filipinas': 'Philippines',
    'Kazajistán': 'Kazakhstan',
    'Irán': 'Iran',
    'Myanmar (Birmania)': 'Myanmar',
    'Uzbekistán': 'Uzbekistan',
    'Benín': 'Benin',
    'Camerún': 'Cameroon',
    'Kenia': 'Kenya',
    'Togo': 'Togo',
    'Ucrania': 'Ukraine',
    'Polonia': 'Poland',
    'Portugal': 'Portugal',
    'Rumania': 'Romania',
    'Trinidad y Tobago': 'Trinidad and Tobago',
    'Afganistán': 'Afghanistan',
    'Pakistán': 'Pakistan',
    'Vietnam': 'Vietnam',
    'Malasia': 'Malaysia',
    'Finlandia': 'Finland',
    'Rusia': 'Russia',
    'Irlanda': 'Ireland',
    'Noruega': 'Norway',
    'Eslovaquia': 'Slovakia',
    'Bélgica': 'Belgium',
    'Bolivia': 'Bolivia',
    'Chile': 'Chile',
    'Jamaica': 'Jamaica',
    'Yemen': 'Yemen',
    'Ghana': 'Ghana',
    'Guinea': 'Guinea',
    'Etiopía': 'Ethiopia',
    'Bulgaria': 'Bulgaria',
    'Kirguistán': 'Kyrgyzstan',
    'Georgia': 'Georgia',
    'Nepal': 'Nepal',
    'Emiratos Árabes Unidos': 'United Arab Emirates',
    'Camboya': 'Cambodia',
    'Uganda': 'Uganda',
    'Lesoto': 'Lesotho',
    'Lituania': 'Lithuania',
    'Suiza': 'Switzerland',
    'Hungría': 'Hungary',
    'Dinamarca': 'Denmark',
    'Haití': 'Haiti',
    'Bielorrusia': 'Belarus',
    'Croacia': 'Croatia',
    'Laos': 'Laos',
    'Baréin': 'Bahrain',
    'Macedonia': 'North Macedonia',
    'República Checa': 'Czech Republic',
    'Sri Lanka': 'Sri Lanka',
    'Zimbabue': 'Zimbabwe',
    'Eritrea': 'Eritrea',
    'Burkina Faso': 'Burkina Faso',
    'Costa Rica': 'Costa Rica',
    'Libia': 'Libya',
    'Barbados': 'Barbados',
    'Tayikistán': 'Tajikistan',
    'Siria': 'Syria',
    'Guadalupe': 'Guadeloupe',
    'Papúa Nueva Guinea': 'Papua New Guinea',
    'Azerbaiyán': 'Azerbaijan',
    'Turkmenistán': 'Turkmenistan',
    'Paraguay': 'Paraguay',
    'Jordania': 'Jordan',
    'Hong Kong': 'Hong Kong',
    'Martinica': 'Martinique',
    'Moldavia': 'Moldova',
    'Qatar': 'Qatar',
    'Mali': 'Mali',
    'Albania': 'Albania',
    'República del Congo': 'Republic of the Congo',
    'Bosnia y Herzegovina': 'Bosnia and Herzegovina',
    'Omán': 'Oman',
    'Túnez': 'Tunisia',
    'Sierra Leona': 'Sierra Leone',
    'Yibuti': 'Djibouti',
    'Burundi': 'Burundi',
    'Montenegro': 'Montenegro',
    'Gabón': 'Gabon',
    'Sudán del Sur': 'South Sudan',
    'Luxemburgo': 'Luxembourg',
    'Namibia': 'Namibia',
    'Mauritania': 'Mauritania',
    'Grecia': 'Greece',
    'Suazilandia': 'Eswatini',
    'Guyana': 'Guyana',
    'Guayana Francesa': 'French Guiana',
    'República Centroafricana': 'Central African Republic',
    'Taiwán': 'Taiwan',
    'Estonia': 'Estonia',
    'Líbano': 'Lebanon',
    'Chipre': 'Cyprus',
    'Guinea-Bissau': 'Guinea-Bissau',
    'Surinam': 'Suriname',
    'Belice': 'Belize',
    'Eslovenia': 'Slovenia',
    'República de Gambia': 'Gambia',
    'Botsuana': 'Botswana',
    'Armenia': 'Armenia',
    'Guinea Ecuatorial': 'Equatorial Guinea',
    'Kuwait': 'Kuwait',
    'Bután': 'Bhutan',
    'Chad': 'Chad',
    'Serbia': 'Serbia',
    'Sáhara Occidental': 'Western Sahara',
    'Italia': 'Italy'
}

cleaned_df['Order Country'] = cleaned_df['Order Country'].replace(country_mapping)

Recategorize product categories

In [15]:
product_category_mapping = {
    'Smart watch': 'Electronics',
    'Perfect Fitness Perfect Rip Deck': 'Sporting Goods',
    "Under Armour Girls' Toddler Spine Surge Runni": "Girls' Apparel",
    "Nike Men's Dri-FIT Victory Golf Polo": "Men's Clothing",
    "Under Armour Men's Compression EV SL Slide": "Men's Footwear",
    "Under Armour Women's Micro G Skulpt Running S": "Women's Apparel",
    "Nike Men's Free 5.0+ Running Shoe": "Men's Footwear",
    "Glove It Women's Mod Oval 3-Zip Carry All Gol": 'Golf Accessories',
    'Bridgestone e6 Straight Distance NFL San Dieg': 'Golf Balls',
    "Columbia Men's PFG Anchor Tough T-Shirt": "Men's Clothing",
    'Titleist Pro V1x Golf Balls': 'Golf Balls',
    'Bridgestone e6 Straight Distance NFL Tennesse': 'Golf Balls',
    'Polar FT4 Heart Rate Monitor': 'Fitness Accessories',
    'ENO Atlas Hammock Straps': 'Camping & Hiking',
    "adidas Men's F10 Messi TRX FG Soccer Cleat": 'Cleats',
    "Brooks Women's Ghost 6 Running Shoe": "Women's Apparel",
    "Nike Men's CJ Elite 2 TD Football Cleat": 'Cleats',
    "Diamondback Women's Serene Classic Comfort Bi": 'Cycling',
    'Industrial consumer electronics': 'Consumer Electronics',
    'Web Camera': 'Consumer Electronics',
    'Dell Laptop': 'Computers',
    'SOLE E25 Elliptical': 'Cardio Equipment',
    'Elevation Training Mask 2.0': 'Boxing & MMA',
    "adidas Men's Germany Black Crest Away Tee": "Men's Clothing",
    'Team Golf Pittsburgh Steelers Putter Grip': 'Golf Accessories',
    'Glove It Urban Brick Golf Towel': 'Golf Accessories',
    'Team Golf Texas Longhorns Putter Grip': 'Golf Accessories',
    "Nike Men's Deutschland Weltmeister Winners Bl": "Men's Clothing",
    'Team Golf St. Louis Cardinals Putter Grip': 'Golf Accessories',
    'Summer dresses': "Women's Clothing",
    'Porcelain crafts': 'Crafts',
    "Men's gala suit": "Men's Clothing",
    'Team Golf Tennessee Volunteers Putter Grip': 'Golf Accessories',
    'Team Golf San Francisco Giants Putter Grip': 'Golf Accessories',
    'Glove It Imperial Golf Towel': 'Golf Accessories',
    "Nike Men's Comfort 2 Slide": "Men's Footwear",
    'Under Armour Hustle Storm Medium Duffle Bag': 'Accessories',
    "Under Armour Kids' Mercenary Slide": "Kids' Footwear",
    "Under Armour Women's Ignite PIP VI Slide": "Women's Footwear",
    "Nike Men's Free TR 5.0 TB Training Shoe": "Men's Footwear",
    'adidas Youth Germany Black/Red Away Match Soc': "Kids' Clothing",
    "TYR Boys' Team Digi Jammer": "Boys' Apparel",
    "Glove It Women's Imperial Golf Glove": 'Golf Gloves',
    'Titleist Pro V1x High Numbers Golf Balls': 'Golf Balls',
    'Bridgestone e6 Straight Distance NFL Carolina': 'Golf Balls',
    "Under Armour Women's Ignite Slide": "Women's Footwear",
    'Titleist Pro V1x High Numbers Personalized Go': 'Golf Balls',
    'GoPro HERO3+ Black Edition Camera': 'Cameras',
    'Total Gym 1400': 'Cardio Equipment',
    "Children's heaters": 'Consumer Electronics',
    'Team Golf New England Patriots Putter Grip': 'Golf Accessories',
    "adidas Kids' F5 Messi FG Soccer Cleat": 'Cleats',
    "Nike Women's Tempo Shorts": "Women's Apparel",
    "Glove It Women's Mod Oval Golf Glove": 'Golf Gloves',
    'Titleist Pro V1 High Numbers Personalized Gol': 'Golf Balls',
    "Under Armour Men's Tech II T-Shirt": "Men's Clothing",
    'Baby sweater': "Children's Clothing",
    'Mio ALPHA Heart Rate Monitor/Sport Watch': 'Fitness Accessories',
    'Field & Stream Sportsman 16 Gun Fire Safe': 'Hunting & Shooting',
    'Sports Books': 'Books',
    "Diamondback Boys' Insight 24 Performance Hybr": 'Cycling',
    'Polar Loop Activity Tracker': 'Fitness Accessories',
    'Garmin Forerunner 910XT GPS Watch': 'Electronics',
    'DVDs': 'DVDs',
    'CDs of rock': 'CDs',
    "Nike Kids' Grade School KD VI Basketball Shoe": "Kids' Footwear",
    "Nike Women's Free 5.0 TR FIT PRT 4 Training S": "Women's Footwear",
    "Hirzl Women's Soffft Flex Golf Glove": 'Golf Gloves',"The North Face Women's Recon Backpack": 'Camping & Hiking',
    'Lawn mower': 'Garden',
    'Nike Dri-FIT Crew Sock 6 Pack': 'Accessories',
    "Nike Women's Legend V-Neck T-Shirt": "Women's Clothing",
    'Garmin Approach S4 Golf GPS Watch': 'Electronics',
    'insta-bed Neverflat Air Mattress': 'Camping & Hiking',
    "Nike Men's Kobe IX Elite Low Basketball Shoe": "Men's Footwear",
    'Adult dog supplies': 'Pet Supplies',
    'First aid kit': 'Health and Beauty',
    'Garmin Approach S3 Golf GPS Watch': 'Electronics',
    'Rock music': 'Music',
    'Fighting video games': 'Video Games',
    'Fitbit The One Wireless Activity & Sleep Trac': 'Fitness Accessories',
    'Stiga Master Series ST3100 Competition Indoor': 'Indoor/Outdoor Games',
    "Diamondback Girls' Clarity 24 Hybrid Bike 201": 'Cycling',
    'adidas Brazuca 2014 Official Match Ball': 'Soccer',
    'GolfBuddy VT3 GPS Watch': 'Electronics',
    'Bushnell Pro X7 Jolt Slope Rangefinder': 'Golf Accessories',
    'Yakima DoubleDown Ace Hitch Mount 4-Bike Rack': 'Cycling',
    "Nike Men's Fingertrap Max Training Shoe": "Men's Footwear",
    'Bowflex SelectTech 1090 Dumbbells': 'Strength Training',
    'SOLE E35 Elliptical': 'Cardio Equipment',
    "Hirzl Women's Hybrid Golf Glove": 'Golf Gloves',
    "Hirzl Men's Hybrid Golf Glove": 'Golf Gloves',
    'TaylorMade 2014 Purelite Stand Bag': 'Golf Bags & Carts',
    'Bag Boy Beverage Holder': 'Accessories',
    'Bag Boy M330 Push Cart': 'Golf Bags & Carts',
    'Clicgear 8.0 Shoe Brush': 'Golf Accessories',
    'Titleist Small Wheeled Travel Cover': 'Golf Bags & Carts',
    'Clicgear Rovic Cooler Bag': 'Golf Accessories',
    'Titleist Club Glove Travel Cover': 'Golf Bags & Carts',
    'Ogio Race Golf Shoes': 'Golf Shoes',
    "LIJA Women's Argyle Golf Polo": "Women's Golf Apparel",
    "LIJA Women's Eyelet Sleeveless Golf Polo": "Women's Golf Apparel",
    "LIJA Women's Button Golf Dress": "Women's Golf Apparel",
    "LIJA Women's Mid-Length Panel Golf Shorts": "Women's Golf Apparel",
    "TaylorMade Women's RBZ SL Rescue": "Women's Golf Clubs",
    "Cleveland Golf Women's 588 RTX CB Satin Chrom": "Women's Golf Clubs",
    "Top Flite Women's 2014 XL Hybrid": "Women's Golf Clubs",
    'MDGolf Pittsburgh Penguins Putter': 'Golf Accessories',
    'TaylorMade White Smoke IN-12 Putter': 'Golf Accessories',
    'Cleveland Golf Collegiate My Custom Wedge 588': 'Golf Accessories',
    "Merrell Men's All Out Flash Trail Running Sho": "Men's Footwear",
    "Merrell Women's Grassbow Sport Waterproof Hik": "Women's Footwear",
    "Merrell Women's Siren Mid Waterproof Hiking B": "Women's Footwear",
    "Merrell Women's Grassbow Sport Hiking Shoe": "Women's Footwear",
    'Toys': 'Toys',
    'Pelican Sunstream 100 Kayak': 'Water Sports',
    'Pelican Maverick 100X Kayak': 'Water Sports',
    "O'Brien Men's Neoprene Life Vest": 'Water Sports'
}

# Use map to update the "Product Category" column based on the "Product Name" column
cleaned_df['Product Category'] = cleaned_df['Product Name'].map(product_category_mapping)
cleaned_df['Product Category'] = cleaned_df['Product Category'].str.strip() # remove space at the end

Converting dates to datetime format

In [16]:
# Converting Date columns to datetime
cleaned_df['Order Date'] = pd.to_datetime(cleaned_df['Order Date'])
cleaned_df = cleaned_df[cleaned_df['Order Date'] < '2017-10-01'] #rationale put in wiki
cleaned_df['Shipping Date'] = pd.to_datetime(cleaned_df['Shipping Date'])
cleaned_df['Order Timestamp'] = cleaned_df['Order Date']

# # Remove the time by setting it to midnight, keeping the column as datetime
# cleaned_df['Order Date'] = cleaned_df['Order Date'].dt.normalize()
# cleaned_df['Shipping Date'] = cleaned_df['Shipping Date'].dt.normalize()

# Extracting the year, month, and day from the 'Order Date' column
cleaned_df['Order Year'] = cleaned_df['Order Date'].dt.year
cleaned_df['Order Month'] = cleaned_df['Order Date'].dt.month_name()
cleaned_df['Order Day'] = cleaned_df['Order Date'].dt.day
cleaned_df['Order Year-Month'] = cleaned_df['Order Date'].dt.to_period('M').astype(str)
cleaned_df['Order Day of Week'] = cleaned_df['Order Date'].dt.day_name()

# Extracting the year, month, and day from the 'Shipping Date' column
cleaned_df['Shipping Year'] = cleaned_df['Shipping Date'].dt.year
cleaned_df['Shipping Month'] = cleaned_df['Shipping Date'].dt.month_name()
cleaned_df['Shipping Day'] = cleaned_df['Shipping Date'].dt.day
cleaned_df['Shipping Year-Month'] = cleaned_df['Shipping Date'].dt.to_period('M').astype(str)
cleaned_df['Shipping Day of Week'] = cleaned_df['Shipping Date'].dt.day_name()

# Remove the time, keeping only the date in 'YYYY-MM-DD' format
cleaned_df['Order Date'] = cleaned_df['Order Date'].dt.date
cleaned_df['Shipping Date'] = cleaned_df['Shipping Date'].dt.date

Annotating or creating columns for downstream analysis

In [17]:
# Annotating late_delivery_risk column
cleaned_df['Late_delivery_risk'] = cleaned_df['Late_delivery_risk'].map({0: 'Not Late', 1: 'Late'})

# Adding difference in shipment days column
cleaned_df['difference in shipment days'] = cleaned_df['Days for shipment (real)'] - cleaned_df['Days for shipment (scheduled)']

# Adding a column of index
cleaned_df['Index'] = range(1, len(cleaned_df) + 1) 

In [18]:
# Update warehouse lat long for warehouse
order_counts = cleaned_df.groupby(['Warehouse Name', 'Warehouse Latitude', 'Warehouse Longitude']).size().reset_index(name='Order Count')
most_orders_location = order_counts.loc[order_counts.groupby('Warehouse Name')['Order Count'].idxmax()]

cleaned_df = pd.merge(cleaned_df.drop(columns=['Warehouse Latitude', 'Warehouse Longitude']), 
                     most_orders_location[['Warehouse Name', 'Warehouse Latitude', 'Warehouse Longitude']], 
                     on='Warehouse Name', 
                     how='left')

In [19]:
has_nan = cleaned_df['Product Category'].isna().any()
print("NaN values in 'Product Category':", has_nan)

NaN values in 'Product Category': False


## Saving cleaned data

In [20]:
display(cleaned_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171962 entries, 0 to 171961
Data columns (total 55 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Transaction Type               171962 non-null  object        
 1   Days for shipment (real)       171962 non-null  int64         
 2   Days for shipment (scheduled)  171962 non-null  int64         
 3   Sales per customer             171962 non-null  float64       
 4   Delivery Status                171962 non-null  object        
 5   Late_delivery_risk             171962 non-null  object        
 6   Product Category               171962 non-null  object        
 7   Customer City                  171962 non-null  object        
 8   Customer Country               171962 non-null  object        
 9   Customer Fname                 171962 non-null  object        
 10  Customer Id                    171962 non-null  int64         
 11  

None

In [21]:
# # Write the DataFrame to a SQL table
# cleaned_df.to_sql('cleaned_order_data', conn, if_exists='replace', index=False)

# 3. Generating stock data

In [22]:
# Function to generate initial stock based on monthly purchase
def generate_initial_stock(total_purchased, restock_buffer=1.2):
    """
    Generates an initial stock level that is always more than the total purchased quantity.
    """
    return int(total_purchased * restock_buffer + fake.random_int(min=0, max=50))

# Function to generate a random restock amount based on the previous month's total purchase
def generate_restock_amount(total_purchased_last_month):
    """
    Generates a random restock number that is more than the total purchased quantity in the previous month.
    """
    return int(total_purchased_last_month + fake.random_int(min=0, max=50))  # Ensure restock is always higher


## Stock data instance

In [23]:
# Query for non-technology products
query = """
SELECT 
    "Warehouse Name", 
    "Product Name", 
    "Warehouse Id", 
    DATE("Order Date") AS "Order Date", 
    "Order Year-Month", 
    "Total Quantity Purchased"
FROM cleaned_order_data
"""

# Execute the query and store the result in a DataFrame
supply_chain_df_aggregated = pd.read_sql_query(query, conn)

# Convert the 'Order Date' and 'Order Year-Month' columns to datetime
supply_chain_df_aggregated['Order Date'] = pd.to_datetime(supply_chain_df_aggregated['Order Date'])
supply_chain_df_aggregated['Order Year-Month'] = supply_chain_df_aggregated['Order Date'].dt.to_period('M')

display(supply_chain_df_aggregated.head())


,Warehouse Name,Product Name,Warehouse Id,Order Date,Order Year-Month,Total Quantity Purchased
0,Apparel,Perfect Fitness Perfect Rip Deck,4,2016-02-24,2016-02,2
1,Golf,Under Armour Girls' Toddler Spine Surge Runni,5,2016-10-25,2016-10,2
2,Golf,Nike Men's Dri-FIT Victory Golf Polo,5,2016-03-30,2016-03,2
3,Golf,Under Armour Girls' Toddler Spine Surge Runni,5,2016-10-30,2016-10,2
4,Golf,Nike Men's Dri-FIT Victory Golf Polo,5,2016-11-28,2016-11,2


In [24]:
# Generating stock data
synthetic_stock = []

# Iterate over unique products and months
for (product_name, month), group in supply_chain_df_aggregated.groupby(['Product Name', 'Order Year-Month']):
    
    # Total quantity purchased in the current month
    total_purchased_this_month = group['Total Quantity Purchased'].sum()

    # Generate initial stock based on the total purchased for the current month
    initial_stock = generate_initial_stock(total_purchased_this_month)

    # Create a new row for each day in the month
    month_start_date = month.start_time
    month_end_date = month.end_time

    # Iterate over each day in the month
    for day in pd.date_range(start=month_start_date, end=month_end_date):
        # Check total quantity sold for that day
        total_sold_today = group.loc[group['Order Date'].dt.date == day.date(), 'Total Quantity Purchased'].sum()
        
        # If it's the first day of the month, restock the inventory
        if day.date() == month_start_date.date():
            # Total quantity sold last month
            previous_month = month - 1
            total_purchased_last_month = supply_chain_df_aggregated.loc[
                (supply_chain_df_aggregated['Product Name'] == product_name) &
                (supply_chain_df_aggregated['Order Year-Month'] == previous_month),
                'Total Quantity Purchased'
            ].sum()

            # Restock the inventory with a restock amount greater than last month's total sales
            restock_amount = generate_restock_amount(total_purchased_last_month)
            current_stock = initial_stock + restock_amount
        else:
            current_stock -= total_sold_today
        
        # Append to the synthetic stock list
        synthetic_stock.append({
            'Product Name': product_name,
            'Date': day,
            'Current Stock': max(current_stock, 0),  # Ensure stock doesn't go negative
            'Total Sold': total_sold_today
        })

# Convert the synthetic stock list to a DataFrame
stock_df = pd.DataFrame(synthetic_stock)



In [25]:
# Check if there are any instances of negative stock
negative_stock_df = stock_df[stock_df['Current Stock'] < 0]
if not negative_stock_df.empty:
    print("Instances of negative stock found:")
    print(negative_stock_df)
else:
    print("No instances of negative stock found.")

No instances of negative stock found.


In [26]:
# Ensure Date is saved in YYYY-MM-DD format
stock_df['Date'] = stock_df['Date'].dt.strftime('%Y-%m-%d')

# Write the DataFrame to a SQL table
# stock_df.to_sql('stock_data', conn, if_exists='replace', index=False)

# 4. Generate Product Data

In [27]:
# Create needed tables 
cleaned_df['Order Date'] = pd.to_datetime(cleaned_df['Order Date'])
cleaned_df = cleaned_df[cleaned_df['Order Date'] < '2017-01-01']
unique_id = cleaned_df[['Product Name', 'Product Id', 'Warehouse Id']].drop_duplicates()

# display(unique_id)

product_price = cleaned_df[['Product Name', 'Product Id', 'Sales', 'Total Quantity Purchased', 'Order Date']]
product_price['Order Date'] =  pd.to_datetime(product_price['Order Date'])
product_price = product_price[product_price['Order Date'] < '2017-01-01']
product_price = product_price.drop(['Order Date'], axis=1)
product_price = product_price.groupby(['Product Id', 'Product Name']).sum().reset_index()
product_price['Price'] = product_price['Sales'] / product_price['Total Quantity Purchased']
product_price = product_price[['Product Id', 'Product Name', 'Price']]

# display(product_price)

/tmp/ipykernel_8/352261937.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_price['Order Date'] =  pd.to_datetime(product_price['Order Date'])


### Create Product Weight, Storage Rate, Storage Cost, Manufacturing Time

In [28]:
product_names = cleaned_df['Product Name'].unique().tolist()
#print(product_names)

random.seed(42)

# Define weight ranges based on keywords
weight_ranges = {
    'Apparel': (0.2, 1.5),
    'Shoes': (0.5, 1.5),
    'Watch': (0.2, 0.7),
    'Electronics': (1, 3),
    'Glove': (0.1, 0.5),
    'Golf': (0.2, 5),
    'Dumbbell': (2, 10),
    'Bike': (5, 20),
    'Kayak': (10, 20),
    'Toy': (0.1, 2),
    'DVD': (0.5, 1),
    'Camera': (0.2, 1),
    'Elliptical': (30, 50),
    'Treadmill': (20, 50),
    'Misc': (0.5, 5)
}

# Function to assign weight based on product name
def assign_weight(product_name):
    for keyword, (min_w, max_w) in weight_ranges.items():
        if keyword.lower() in product_name.lower():
            return round(random.uniform(min_w, max_w), 2)
    return round(random.uniform(0.5, 5), 2)  # Default weight if no match

# Create a DataFrame
df = pd.DataFrame({'Product Name': product_names})
df['Weight (kg)'] = df['Product Name'].apply(assign_weight)

print(df)


                                     Product Name  Weight (kg)
0                Perfect Fitness Perfect Rip Deck         3.38
1   Under Armour Girls' Toddler Spine Surge Runni         0.61
2            Nike Men's Dri-FIT Victory Golf Polo         1.52
3      Under Armour Men's Compression EV SL Slide         1.50
4   Under Armour Women's Micro G Skulpt Running S         3.81
5               Nike Men's Free 5.0+ Running Shoe         3.55
6   Glove It Women's Mod Oval 3-Zip Carry All Gol         0.46
7   Bridgestone e6 Straight Distance NFL San Dieg         0.89
8      adidas Men's F10 Messi TRX FG Soccer Cleat         2.40
9         Nike Men's CJ Elite 2 TD Football Cleat         0.63
10  Diamondback Women's Serene Classic Comfort Bi         1.48
11      adidas Men's Germany Black Crest Away Tee         2.77
12      Team Golf Pittsburgh Steelers Putter Grip         0.33
13                Glove It Urban Brick Golf Towel         0.18
14          Team Golf Texas Longhorns Putter Grip      

In [29]:
df = df.merge(unique_id[['Product Name', 'Product Id', 'Warehouse Id']], on='Product Name', how='left')
df['Daily Storage Rate'] = 0.02
df['Daily Storage Cost'] = df['Weight (kg)'] * df['Daily Storage Rate']
df['Manufacturing Time'] = df.apply(
    lambda x: np.random.randint(6, 11) if x['Warehouse Id'] == 10 else
              np.random.randint(3, 7) if x['Warehouse Id'] in [4, 3] else
              np.random.randint(3, 11),
    axis=1
)
# display(df)

### Create Order Price

In [30]:
wholesale_discount = {
    'Apparel': 0.6,
    'Shoe': 0.6,
    'Watch': 0.4,
    'Electronics': 0.4,
    'Glove': 0.4,
    'Golf': 0.5,
    'Dumbbell': 0.3,
    'Bike': 0.5,
    'Kayak': 0.5,
    'Toy': 0.6,
    'DVD': 0.4,
    'Camera': 0.4,
    'Elliptical': 0.3,
    'Treadmill': 0.3,
    'Misc': 0.4,
    'Soccer': 0.5,
    'Slide': 0.6,
    'Sock': 0.6,
    'Short': 0.6,
    'Shirt': 0.6,
    'Bag': 0.6,
    'Running': 0.3,
    'Deck': 0.3,
    'Football': 0.5,
    'Weltmeister': 0.6,
    'Germany': 0.6,
    'Hybrid': 0.6,
    'Jammer': 0.5,
    'Toddler': 0.6,
    'Personalized': 0.5,
    'Bridgestone': 0.5,
    'Gun': 0.5,
    'Vest': 0.5,
    'Hammock': 0.6,
    'Serene': 0.5
    
}

def calculate_order_price(row):
    category = row['Product Name']
    sale_price = row['Price']
    for keyword, disc in wholesale_discount.items():
        if keyword.lower() in category.lower():
            discount = disc
            order_price = sale_price * (1 - discount)
            return round(order_price, 2)

# Apply the order price calculation to the products DataFrame
product_price['order_price'] = product_price.apply(calculate_order_price, axis=1)
display(product_price)

,Product Id,Product Name,Price,order_price
0,37,adidas Kids' F5 Messi FG Soccer Cleat,34.990001,17.50
1,44,adidas Men's F10 Messi TRX FG Soccer Cleat,59.990002,30.00
2,93,Under Armour Men's Tech II T-Shirt,24.990000,10.00
3,116,Nike Men's Comfort 2 Slide,44.990001,18.00
4,134,Nike Women's Legend V-Neck T-Shirt,25.000000,10.00
5,135,Nike Dri-FIT Crew Sock 6 Pack,22.000000,8.80
6,172,Nike Women's Tempo Shorts,30.000000,12.00
7,191,Nike Men's Free 5.0+ Running Shoe,99.990000,40.00
8,235,Under Armour Hustle Storm Medium Duffle Bag,34.990001,14.00
9,249,Under Armour Women's Micro G Skulpt Running S,54.970001,38.48


In [31]:
product_price.rename(columns={'order_price': 'Manufacturing Cost'}, inplace=True)
product_info_final = df.merge(product_price[['Product Id', 'Manufacturing Cost', 'Price']], on='Product Id', how='left')
product_info_final = product_info_final.dropna()
display(product_info_final)

,Product Name,Weight (kg),Product Id,Warehouse Id,Daily Storage Rate,Daily Storage Cost,Manufacturing Time,Manufacturing Cost,Price
0,Perfect Fitness Perfect Rip Deck,3.38,365,4,0.02,0.0676,5,41.99,59.990002
1,Under Armour Girls' Toddler Spine Surge Runni,0.61,627,5,0.02,0.0122,3,16.00,39.990001
2,Nike Men's Dri-FIT Victory Golf Polo,1.52,502,5,0.02,0.0304,5,25.00,50.000000
3,Under Armour Men's Compression EV SL Slide,1.50,278,3,0.02,0.0300,4,18.00,44.990001
4,Under Armour Women's Micro G Skulpt Running S,3.81,249,3,0.02,0.0762,3,38.48,54.970001
5,Nike Men's Free 5.0+ Running Shoe,3.55,191,3,0.02,0.0710,3,40.00,99.990000
6,Glove It Women's Mod Oval 3-Zip Carry All Gol,0.46,917,6,0.02,0.0092,7,13.19,21.990000
7,Bridgestone e6 Straight Distance NFL San Dieg,0.89,828,6,0.02,0.0178,10,15.99,31.990000
8,adidas Men's F10 Messi TRX FG Soccer Cleat,2.40,44,2,0.02,0.0480,5,30.00,59.990002
9,Nike Men's CJ Elite 2 TD Football Cleat,0.63,403,4,0.02,0.0126,4,65.00,129.990005


### Save and Store product information table into database

In [32]:
# Write the DataFrame to a SQL table
# product_info_final.to_sql('product_info', conn, if_exists='replace', index=False)

# Database Sanity Check

In [33]:
# Query for non-technology products
query = """
SELECT * 
FROM cleaned_order_data
"""

# Execute the query and store the result in a DataFrame
x = pd.read_sql_query(query, conn)
display(x.head())

,Transaction Type,Days for shipment (real),Days for shipment (scheduled),Sales per customer,Delivery Status,Late_delivery_risk,Product Category,Customer City,Customer Country,Customer Fname,Customer Id,Customer Lname,Customer Segment,Customer State,Customer Street,Warehouse Id,Warehouse Name,Market,Order City,Order Country,Order Date,Order Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Profit Ratio,Total Quantity Purchased,Sales,Order Item Total,Order Profit,Order Region,Order State,Order Status,Product Id,Product Category Id,Product Name,Product Price,Product Status,Shipping Date,Shipping Mode,Order Timestamp,Order Year,Order Month,Order Day,Order Year-Month,Order Day of Week,Shipping Year,Shipping Month,Shipping Day,Shipping Year-Month,Shipping Day of Week,difference in shipment days,Index,Warehouse Latitude,Warehouse Longitude
0,PAYMENT,5,2,115.180000,Late delivery,Late,Sporting Goods,Bayamon,Puerto Rico,Mary,9083,Frank,Home Office,PR,75 Sunny Grounds,4,Apparel,Pacific Asia,Mirzapur,India,2016-02-24,28744,4.8,0.04,71956,-0.27,2,119.980003,115.180000,-30.750000,South Asia,Uttar Pradesh,PENDING_PAYMENT,365,17,Perfect Fitness Perfect Rip Deck,59.990002,0,2016-02-29,Second Class,2016-02-24 13:57:00,2016,February,24,2016-02,Wednesday,2016,February,29,2016-02,Monday,3,1,39.495914,-98.989983
1,PAYMENT,2,2,79.180000,Shipping on time,Not Late,Girls' Apparel,Caguas,Puerto Rico,Mary,4741,Smith,Home Office,PR,9731 Honey Fox Towers,5,Golf,Pacific Asia,Bursa,Turkey,2016-10-25,45461,0.8,0.01,113598,-1.55,2,79.980003,79.180000,-122.730003,West Asia,Bursa,PENDING_PAYMENT,627,29,Under Armour Girls' Toddler Spine Surge Runni,39.990002,0,2016-10-27,Second Class,2016-10-25 14:39:00,2016,October,25,2016-10,Tuesday,2016,October,27,2016-10,Thursday,0,2,39.495914,-98.989983
2,PAYMENT,6,2,96.000000,Late delivery,Late,Men's Clothing,Caguas,Puerto Rico,Elizabeth,639,Pittman,Home Office,PR,7573 Golden Treasure Centre,5,Golf,Pacific Asia,Murray Bridge,Australia,2016-03-30,31115,4.0,0.04,77757,0.35,2,100.000000,96.000000,33.599998,Oceania,Australia del Sur,PENDING_PAYMENT,502,24,Nike Men's Dri-FIT Victory Golf Polo,50.000000,0,2016-04-05,Second Class,2016-03-30 04:37:00,2016,March,30,2016-03,Wednesday,2016,April,5,2016-04,Tuesday,4,3,39.495914,-98.989983
3,PAYMENT,2,2,75.980003,Shipping on time,Not Late,Girls' Apparel,Caguas,Puerto Rico,Katherine,9702,Tyler,Home Office,PR,8369 Sunny Crossing,5,Golf,Pacific Asia,Kartal,Turkey,2016-10-30,45766,4.0,0.05,114401,0.33,2,79.980003,75.980003,24.690001,West Asia,Estambul,PENDING_PAYMENT,627,29,Under Armour Girls' Toddler Spine Surge Runni,39.990002,0,2016-11-01,Second Class,2016-10-30 01:31:00,2016,October,30,2016-10,Sunday,2016,November,1,2016-11,Tuesday,0,4,39.495914,-98.989983
4,PAYMENT,3,2,91.000000,Late delivery,Late,Men's Clothing,Caguas,Puerto Rico,Mary,9114,Smith,Home Office,PR,1425 Fallen Fox Arbor,5,Golf,Pacific Asia,Ulan Bator,Mongolia,2016-11-28,47752,9.0,0.09,119405,0.10,2,100.000000,91.000000,9.100000,Eastern Asia,Ulán Bator,PENDING_PAYMENT,502,24,Nike Men's Dri-FIT Victory Golf Polo,50.000000,0,2016-12-01,Second Class,2016-11-28 01:18:00,2016,November,28,2016-11,Monday,2016,December,1,2016-12,Thursday,1,5,39.495914,-98.989983


In [34]:
def create_data_dictionary(x):
    data_dict = {}
    for col in x.columns:
        data_dict[col] = {
            'Data Type': x[col].dtype,
            'Number of Unique Values': x[col].nunique(),
            'Number of Missing Values': x[col].isnull().sum(),
            'Sample Values': x[col].unique()[:5],  # Show up to 5 unique values
        }
        
        # Add summary stats for numeric columns
        if pd.api.types.is_numeric_dtype(x[col]):
            data_dict[col].update({
                'Mean': x[col].mean(),
                'Min': x[col].min(),
                'Max': x[col].max()
            })
    
    return data_dict

# Generate the data dictionary
data_dictionary = create_data_dictionary(x)

# Display the data dictionary
for col, details in data_dictionary.items():
    print(f"Column: {col}")
    for key, value in details.items():
        print(f"  {key}: {value}")
    print()  # Add a blank line between columns

Column: Transaction Type
  Data Type: object
  Number of Unique Values: 4
  Number of Missing Values: 0
  Sample Values: ['PAYMENT' 'TRANSFER' 'DEBIT' 'CASH']

Column: Days for shipment (real)
  Data Type: int64
  Number of Unique Values: 7
  Number of Missing Values: 0
  Sample Values: [5 2 6 3 4]
  Mean: 3.497249392307603
  Min: 0
  Max: 6

Column: Days for shipment (scheduled)
  Data Type: int64
  Number of Unique Values: 4
  Number of Missing Values: 0
  Sample Values: [2 4 1 0]
  Mean: 2.931944266756609
  Min: 0
  Max: 4

Column: Sales per customer
  Data Type: float64
  Number of Unique Values: 2601
  Number of Missing Values: 0
  Sample Values: [115.1800003   79.18000031  96.          75.98000336  91.        ]
  Mean: 178.95950220674723
  Min: 7.489999771
  Max: 1699.98999

Column: Delivery Status
  Data Type: object
  Number of Unique Values: 4
  Number of Missing Values: 0
  Sample Values: ['Late delivery' 'Shipping on time' 'Advance shipping' 'Shipping canceled']

Column: Lat

In [35]:
# close connection
conn.close()